In [6]:
import pandas as pd

# read password and user to database
credentials_file = r'./credentials.pw'
credentials = pd.read_csv(credentials_file, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]


writer heiko


In [39]:
11%11

0

In [51]:
numbers = [ii for ii in range(1,101)]
primes = []

def isPrime(number):
    rest = []
    for jj in range(2,number):
        rest.append(number%jj)
    if 0 in rest:
        return False
    else: 
        return True
            

for n in numbers:
    if isPrime(n) == 1:
        primes.append(n)

In [52]:
primes

[1,
 2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

In [73]:
import pandas as pd
import pymysql


# read password and user to database
credentials_file = r'./credentials.pw'

credentials = pd.read_csv(credentials_file, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]


def insert_into_DB(tableName, lstCols, lstValues, username, password):
	"""
	Inserts a new row in the database in the table tableName. The column names should be in the form of a list and the respective
	values should also be a list in the order of the columns to be updated into.
	"""

	# connect to the database
	db = pymysql.connect(host="twofast-RPi3-0",  # your host
					 user=username,  # username
					 passwd=password,  # password
					 db="NG_twofast_DB", # name of the database
					 charset='utf8',
					 cursorclass=pymysql.cursors.DictCursor)
	cur = db.cursor()

	# SQL statement execution
	# Assert that the length of the columns matches the length of the values
	assert len(lstCols) == len(lstValues)

	# IF only power or frequency to update, i.e. one value
	if len(lstCols) == 1:
		try:
			cur.execute("INSERT INTO %(tablename)s (%(col)s) VALUES (%(val)s)" % {"tablename": tableName, "col": lstCols[0], "val":lstValues[0]})
		except:
			cur.rollback()
	else:
	# more than one value to update
		try:
			cur.execute("INSERT INTO %(tablename)s (%(col1)s, %(col2)s) VALUES (%(val1)s, %(val2)s)" % {"tablename": tableName, "col1": lstCols[0], "col2": lstCols[1], "val1":lstValues[0], "val2":lstValues[1]})
		except:
			cur.rollback()
# cursor.execute("SELECT * FROM %(tablename)s" % {"tablename": "activity"})

	db.commit()
	cur.close()

tableName = 'microwave_generator_temperature'
lstCols = ['temperature1', 'temperature2']
lstValues = ['10', '30']

insert_into_DB(tableName, lstCols, lstValues, user, pw)

In [135]:
# read password and user to database
credentials_file = r'./credentials.pw'

credentials = pd.read_csv(credentials_file, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]

db = pymysql.connect(host="twofast-RPi3-0",  # your host
                 user=user,  # username
                 passwd=pw,  # password
                 db="NG_twofast_DB", # name of the database
                 charset='utf8',
                 cursorclass=pymysql.cursors.DictCursor)
cur = db.cursor()

try:
    cur.execute("INSERT INTO microwave_generator_control (command, executed) VALUES ('a 3 test', 0)")
except:
    cur.rollback()

# cursor.execute("SELECT * FROM %(tablename)s" % {"tablename": "activity"})

db.commit()
cur.close()


In [137]:
mysql_connection = pymysql.connect(host="twofast-RPi3-0",  # your host
                 user=user,  # username
                 passwd=pw,  # password
                 db="NG_twofast_DB", # name of the database
                 charset='utf8',
                 cursorclass=pymysql.cursors.DictCursor)

def getCommandsToExecute(mysql_connection):
    """
    Get the last 10 commands that were not executed
    """
    query = "SELECT * FROM microwave_generator_control WHERE executed = 0 ORDER BY time_created ASC LIMIT 10"
    df = pd.read_sql(query, mysql_connection)


    return df

df = getCommandsToExecute(mysql_connection)
for index, row in df.iterrows():
    print(row['command'], row['id'])

a 1 test 1
a 2 test 2
a 3 test 3


In [116]:
def updateCommandAsExecuted(command_id, timeExecuted):
    """
    After the command was sent to the microwave generator (and no error returned), update in the database that the command has been sent.
    """
    timeExecuted = timeExecuted.strftime('%Y-%m-%d %H:%M:%S')
    cur = db.cursor()
    try:
        cur.execute("UPDATE microwave_generator_control SET executed = 1, time_executed = '%(time)s' WHERE id = %(commandId)s" % {"time": timeExecuted, "commandId": command_id })
    except:
        cur.rollback()

    db.commit()
    cur.close()


command_id = 1
import datetime
timeExecuted = datetime.datetime.now()
updateCommandAsExecuted(command_id, timeExecuted)

In [107]:
import datetime
s = datetime.datetime.now()
import time    
s.strftime('%Y-%m-%d %H:%M:%S')

'2019-06-27 16:44:20'